In [10]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import sys
print(sys.executable)
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path, models_path
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import matplotlib.pyplot as plt
from FCMAE_pytorch_updated import FCMAE


/home/nwertheim/miniconda3/bin/python


ModuleNotFoundError: No module named 'FCMAE_pytorch_updated'

In [4]:
save_dir = os.path.join(univariate_data_path, "target_univariate.npy")
target_data = np.load(save_dir, allow_pickle=True)
target_data = [item for item in target_data if item['preterm'] is not None]



In [5]:
print(target_data[0])

{'record_name': np.str_('Hopper-2023_05_25_12_03_38-0000010090-0003'), 'signal': array([[-0.70613831],
       [-0.9054771 ],
       [-0.7669727 ],
       ...,
       [-0.4010203 ],
       [-0.44152144],
       [-0.37364218]]), 'fs': 20, 'preterm': 0}


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

class ClassifierWithEncoder(nn.Module):
    def __init__(self, encoder, input_dim, hidden_dim, output_dim):
        super(ClassifierWithEncoder, self).__init__()
        self.encoder = encoder  # Pre-trained encoder (frozen)
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        # Pass input through the frozen encoder
        with torch.no_grad():
            encoded_features = self.encoder(x)  # Freeze encoder during training

        # Pass the encoded features through the classification head (MLP)
        output = self.mlp(encoded_features)
        return output


In [7]:
import numpy as np

def create_time_windows_with_labels(data, window_length=12000, step_size=12000):
    """
    Create sliding windows of a specified length and step size from a list of signals,
    along with their associated labels (preterm).

    Args:
        data: list of dicts, each containing a 'signal' array of shape (seq_len, 1)
        window_length: number of time steps in each window
        step_size: number of time steps to move between windows (for overlap)

    Returns:
        windows: np.ndarray of shape (num_windows, window_length), univariate windows
        labels: np.ndarray of shape (num_windows,), preterm labels (0 or 1)
    """
    windows = []
    labels = []

    for entry in data:
        signal = entry['signal'].flatten()  # shape: (seq_len,)
        label = entry['preterm']  # Preterm label
        signal_length = len(signal)

        if signal_length < window_length:
            continue

        for start_idx in range(0, signal_length - window_length + 1, step_size):
            end_idx = start_idx + window_length
            window = signal[start_idx:end_idx]
            windows.append(window)
            labels.append(label)

    return np.array(windows), np.array(labels)


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Example dataset with windows and labels
windows, labels = create_time_windows_with_labels(target_data)

# Convert to tensors
windows_tensor = torch.tensor(windows, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

# Create DataLoader
batch_size = 32
train_dataset = TensorDataset(windows_tensor, labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Load the pre-trained encoder
encoder = FCMAE(dim=1, num_blocks=3)  # Ensure to match the encoder architecture
encoder.load_state_dict(torch.load(path))  # Load the pre-trained encoder's weights
encoder.eval()  # Set encoder to eval mode (no gradients)

# Freeze the encoder weights
for param in encoder.parameters():
    param.requires_grad = False

# Define the classifier model
model = ClassifierWithEncoder(encoder, input_dim=12000, hidden_dim=256, output_dim=2)

# Move the model to the device
model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()  # For binary classification (0 vs. 1)
optimizer = optim.Adam(model.mlp.parameters(), lr=1e-3)  # Only optimize the MLP

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for batch in train_loader:
        batch_input, batch_labels = batch
        batch_input, batch_labels = batch_input.to(device), batch_labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        output = model(batch_input)
        
        # Compute the loss
        loss = criterion(output, batch_labels)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Print average loss every epoch
    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")


NameError: name 'FCMAE' is not defined